## Import the required libraries

In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import folium

/home/framsses/Documents/caffenio_location/coffee/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Create the GeoDataFrame of Hermosillo
First, We read the file where the information was located. Then, We filtered the data to get the information of Hermosillo. We also transformed the polygon type to manage decimals.

In [3]:
son_df = gpd.read_file('data/26sonora/26-SONORA/COLONIA.dbf')
hillo_df = son_df[son_df.MUNICIPIO == 49]
hillo_df.geometry = hillo_df.geometry.to_crs(epsg=4326) # to manage decimals
hillo_df.head(3)


/home/framsses/Documents/caffenio_location/coffee/lib/python3.10/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,ID,ENTIDAD,MUNICIPIO,NOMBRE,CLASIFICAC,CP,OTROS_CP,CONTROL,Geometry1_,geometry
1291,1292,26,49,OPATAS,2,83104,None,6228,"1=X,z_Y","POLYGON ((-111.01043 29.15603, -111.01040 29.1..."
1292,1293,26,49,PUEBLO ALEGRE,2,83104,None,1346,"1=X,z_Y","POLYGON ((-111.00755 29.15346, -111.01061 29.1..."
1293,1294,26,49,TUBAC,2,83104,None,1347,"1=X,z_p","POLYGON ((-111.01043 29.15603, -111.01125 29.1..."


## Get the locations from the web page in order to convert it.
In this case, We scrapped the data from the web page of the bussiness. Then, We identified the places of interest.
We also used the shapely library to convert the information into a Point Geometry Type. For that goal, We worked with the lat and long.

In [4]:
coffee_loc = pd.read_csv('data/hillo_coffee_loc.csv').drop(columns=['id'])
coffee_loc['geometry'] = coffee_loc.apply(lambda row: Point((row.lng, row.lat)), axis=1)
coffee_loc.head()

/home/framsses/Documents/caffenio_location/coffee/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,name,lat,lng,geometry
0,Planta,29.114423,-110.912576,POINT (-110.9125764309198 29.114422977433104)
1,Girasol,29.102622,-110.992914,POINT (-110.99291392051327 29.10262242023761)
2,Dila,29.115971,-110.952745,POINT (-110.95274515982835 29.11597058609762)
3,Progreso,29.147010,-110.966135,POINT (-110.96613474672334 29.147009621422217)
4,Kino,29.099199,-110.939072,POINT (-110.93907176812294 29.099198883299785)


## Create the GeoDataFrame of the coffee shops
We used GeoDataFrame to transform the data. We define the epsg as 4326 to work with decimals.

In [5]:
hillo_coffee_crs = 'epsg:4326'
hillo_coffee_geo = gpd.GeoDataFrame(coffee_loc,
crs = hillo_coffee_crs,
geometry = coffee_loc.geometry)
hillo_coffee_geo.head()

,name,lat,lng,geometry
0,Planta,29.114423,-110.912576,POINT (-110.91258 29.11442)
1,Girasol,29.102622,-110.992914,POINT (-110.99291 29.10262)
2,Dila,29.115971,-110.952745,POINT (-110.95275 29.11597)
3,Progreso,29.147010,-110.966135,POINT (-110.96613 29.14701)
4,Kino,29.099199,-110.939072,POINT (-110.93907 29.09920)


## Joined the data of Hermosillo and the coffee shops locations
We intersected the data to get the suburbs where there are coffe shops in Hermosillo.

In [6]:
hillo_coffee_gdf = gpd.sjoin(hillo_df, hillo_coffee_geo, predicate='intersects')
hillo_coffee_gdf.head(2)

,ID,ENTIDAD,MUNICIPIO,NOMBRE,CLASIFICAC,CP,OTROS_CP,CONTROL,Geometry1_,geometry,index_right,name,lat,lng
1324,1325,26,49,SAN BOSCO,2,83170,None,1380,"1=X,zG","POLYGON ((-111.03149 29.12198, -111.03025 29.1...",28,GasparLuken,29.121814,-111.030934
1393,1394,26,49,RAQUET CLUB,2,83200,None,1451,"1=X,ziF","POLYGON ((-110.99155 29.09145, -110.99147 29.0...",5,Navarrete,29.090023,-110.986734


## Get the total coffee shops by suburb

In [7]:
coffee_counts = hillo_coffee_gdf.groupby('NOMBRE').index_right.count().reset_index(level=0)
coffee_counts.columns = ['NOMBRE','TOTAL']
coffee_counts.head()

,NOMBRE,TOTAL
0,AKIWIKI,1
1,ANTARA RESIDENCIAL,1
2,ASTURIAS RESIDENCIAL,1
3,BUGAMBILIAS,1
4,CENTENARIO,1


In [8]:
hillo_coffee_counts = pd.merge(hillo_coffee_gdf, coffee_counts, on='NOMBRE')
hillo_coffee_counts.head(2)

,ID,ENTIDAD,MUNICIPIO,NOMBRE,CLASIFICAC,CP,OTROS_CP,CONTROL,Geometry1_,geometry,index_right,name,lat,lng,TOTAL
0,1325,26,49,SAN BOSCO,2,83170,None,1380,"1=X,zG","POLYGON ((-111.03149 29.12198, -111.03025 29.1...",28,GasparLuken,29.121814,-111.030934,1
1,1394,26,49,RAQUET CLUB,2,83200,None,1451,"1=X,ziF","POLYGON ((-110.99155 29.09145, -110.99147 29.0...",5,Navarrete,29.090023,-110.986734,1


## Show a Map of Hermosillo, Sonora
In order to do this, We used folium. We center the map with the information provided for the business.
The coords to center the data were, lat: 29.098, lng: -110.950.

In [9]:
hillo_map = folium.Map(location=[29.098, -110.950], zoom_start=12)

display(hillo_map)

## Add characteristics to create an interactive Choropleth Map
Choropleth can be easily created by binding the data between Pandas DataFrames/Series and Geo/TopoJSON geometries.

In [14]:
hillo_map.choropleth(
    geo_data=hillo_coffee_counts,
    name='geometry',
    data=hillo_coffee_counts,
    columns=['NOMBRE','TOTAL'],
    key_on='feature.properties.NOMBRE',
    fill_color='OrRd',
    fill_opacity=0.75,
    line_opacity=0.5,
    legend_name='Number of Coffee Shops by Suburb'
)

/home/framsses/Documents/caffenio_location/coffee/lib/python3.10/site-packages/folium/folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


## Show the Interactive Choropleth Map

In [15]:
folium.LayerControl().add_to(hillo_map)
display(hillo_map)

## Save map as an html file

In [ ]:
hillo_map.save('coffee_hillo.html')